In [ ]:
"""
Created on Mon Jul 19 17:26:18 2021

@author: 4ka
"""
import sys

import gurobipy as gp
import numpy as np
import torch
import torch.nn.functional as F
from gurobipy import GRB

In [ ]:
# import my functions
from gurobi_ml.pytorch import Sequential

In [ ]:
# Load data
X = torch.from_numpy(np.genfromtxt("X.csv")).float()
Y = torch.from_numpy(np.genfromtxt("Y.csv")).float()

In [ ]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return X.shape[0]

    def __getitem__(self, i):
        return self.X[i, :], self.y[i]

In [ ]:
hs = 128
torch.manual_seed(10)
# Define a simple sequential network
model = torch.nn.Sequential(
    torch.nn.Linear(X.shape[1], hs),
    torch.nn.ReLU(),
    torch.nn.Linear(hs, hs),
    torch.nn.ReLU(),
    torch.nn.Linear(hs, hs),
    torch.nn.ReLU(),
    torch.nn.Linear(hs, Y.shape[1]),
)

In [ ]:
# Construct our loss function and an Optimizer.
criterion = torch.nn.MSELoss()
dataloader = torch.utils.data.DataLoader(MyData(X, Y), batch_size=100, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
for t in range(100):
    size = len(dataloader.dataset)
    for batch, (feat, target) in enumerate(dataloader):
        # Zero gradients
        optimizer.zero_grad()
        # Forward pass: Compute predicted y by passing x to the model
        pred = model(feat)
        # Compute and print loss
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        target = model(X)
        # Compute and print loss
        loss = criterion(target, Y)
        print(f"Epoch {t} loss: {loss.item():>7f}")

In [ ]:
p = np.random.randint(30, size=24)

In [ ]:
m = gp.Model()
x = m.addMVar((1, 48), vtype=GRB.CONTINUOUS, name="x", lb=0, ub=410)
y = m.addMVar((1, 24), lb=20, ub=30, vtype=GRB.CONTINUOUS, name="y")
m.setObjective(p @ x[0, 24:], GRB.MINIMIZE)
# Add constraint to predict value of y using x
nn2gurobi = Sequential(m, model, x, y)

In [ ]:
m.optimize()

In [ ]:
print(f"Maximal error in predicted values in solution {np.max(np.abs(nn2gurobi.get_error()))}")

In [ ]:
y.X